In [1]:
import pandas as pd
import requests


In [2]:
deputies = pd.read_csv("data/deputados.csv")
dep_ids = list(deputies.id_dep)

In [3]:
url = "https://dadosabertos.camara.leg.br/api/v2/deputados?{}&idLegislatura=55&itens={}"
headers = {'accept': 'application/json'}

In [4]:
def get_deputies_detail(dep_ids, id_name,bulk,to_log):
    all_response = []
    no_answer = set()
    for i in range(0, len(dep_ids), bulk):
        if to_log:
            print(i, len(dep_ids))
        selected_dep_ids = dep_ids[i:i+bulk]
        ids_string = "&".join(["{}={}".format(id_name, id) for id in selected_dep_ids])
        r = requests.get(url.format(ids_string, bulk), headers=headers)
        response = r.json()
        all_response += response['dados']
        if len(response['dados']) != len(selected_dep_ids):
            no_answer |= (set(selected_dep_ids) - set([r['id'] for r in response['dados']]))
    return all_response, list(no_answer)

def get_by_name(names):
    all_response = []
    no_answer = set()
    for cur_name in names:
        ids_string = "nome={}".format(cur_name)
        r = requests.get(url.format(ids_string, bulk), headers=headers)
        response = r.json()
        all_response += response['dados']
        if len(response['dados']) == 0:
            no_answer.add(cur_name)
    return all_response, list(no_answer)

In [5]:
all_response, no_answer = get_deputies_detail(dep_ids, "id", 20, False)

In [6]:
no_answer

[]

In [7]:
names = list(deputies[deputies.id_dep.isin(no_answer)].nome)
assert len(names) == len(no_answer)

In [8]:
all_response_names, no_answer_names = get_by_name(names)

In [9]:
assert len(no_answer_names) == 0
assert len(all_response + all_response_names) == len(dep_ids)

In [18]:
data_by_api = pd.DataFrame(all_response + all_response_names)

# .to_csv("data/deputados_info_by_api.csv",index=False, encoding="utf8")

In [24]:
change = {"pode": "podemos", "sd": "solidaried"}
data_by_api.siglaPartido = data_by_api.siglaPartido.apply(lambda x: x.lower())
data_by_api.siglaPartido = data_by_api.siglaPartido.apply(lambda x: change.get(x, x))
data_by_api.to_csv("data/deputados_info_by_api.csv",index=False, encoding="utf8")

In [29]:
data_by_api[data_by_api.siglaPartido == 'ppl']

,id,idLegislatura,nome,siglaPartido,siglaUf,uri,uriPartido,urlFoto
558,178863,55,ULDURICO JUNIOR,ppl,BA,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
